In [ ]:
data = {'Task001_LiverTumor':
        {'101.nii': [
            {'slice': {'ax': 'z', 'frame': 505}, 'alg1': '2-CE', 'alg2': 'Focal'},
            {'slice': {'ax': 'z', 'frame': 505}, 'alg1': 'Iou', 'alg2': 'Focal'},
            {'slice': {'ax': 'z', 'frame': 505}, 'alg1': '2-CE', 'alg2': 'pGDice'},
            {'slice': {'ax': 'z', 'frame': 505}, 'alg1': '2-CE', 'alg2': 'pGDice'},
        ],
            '13.nii': [
                {'slice': {'ax': 'x', 'frame': 136}, 'alg1': 'pCE', 'alg2': 'TopK10'},
        ],
            '108.nii': [
                {'slice': {'ax': 'x', 'frame': 176}, 'alg1': 'DiceFocal', 'alg2': 'pGDice'},
                {'slice': {'ax': 'z', 'frame': 535}, 'alg1': 'Tversky', 'alg2': 'pGDice'},
        ],
        }

        }
root = '../datasets'
out_dir = '../out/'


In [ ]:
import evalseg
evalseg.reload()
import glob,os
import matplotlib.pyplot as plt
for dataset_name in data:
    dataset_data=data[dataset_name]
    dataset=evalseg.io.NibDataset(f'{root}/{dataset_name}')
    for case in dataset_data:
        # dataset.load_all_of_case(case)
        gto,gto_voxelsize=dataset.get_groundtruth(case)
        cto,_=dataset.get_CT(case)

        for expr in dataset_data[case]:
            ax,frame=expr['slice']['ax'], expr['slice']['frame']
            gt, voxelsize = evalseg.geometry.slice(gto, gto_voxelsize, ax,[frame])
            ct, _ = evalseg.geometry.slice(cto, None, ax,[frame])
            mme = evalseg.metrics.MultiClassMetric(
                    evalseg.metrics.MME,
                    dataset.num_labels
                )
            mme.set_reference(gt, voxelsize)
            preds={
                d:dataset.get_prediction(d, case)[0]
                for d in [expr['alg1'],expr['alg2']]
            }
            preds={
                d:evalseg.geometry.slice(preds[d], None, ax, [frame])[0]
                for d in preds
            }
            
            ctlbl = f"{dataset_name.split('_')[1]} {case} {ax}={frame}"
            out_path=f'{out_dir}/item-{dataset_name}-{case}-{ax}-{frame}'
            evalseg.ui.multi_plot_2d(ct, gt, preds, spacing=voxelsize, ctlbl= ctlbl, 
            z_titles= [f'{dataset_name}-{case}-{ax}-{frame}'],dst=f'{out_path}-allct.png',show=False)

            mme = evalseg.metrics.MultiClassMetric(evalseg.metrics.MME,dataset.num_labels)
            mme.set_reference(gt, voxelsize)
            res = mme.evaluate_multi(preds)
            evalseg.ui.plot_metric_multi(res, name='',dst=f'{out_path}-metric.png',show=False)
            img=evalseg.ui.img.concat(glob.glob(f'{out_path}-[!res]*.png'), out=f'{out_path}-res.png')
            [os.remove(f) for f in glob.glob(f'{out_path}-[!res]*.png')]
            
            fig,ax=plt.subplots(1,1,dpi=400);ax.set_axis_off();ax.imshow(img);plt.show()
        
    